In [2]:
import os

In [3]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl'

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 param_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config



        


In [13]:
import os
import zipfile
import gdown
from cnnclassifier import logger
from cnnclassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):

        try:
            dataset_url =self.config.source_URL
            zip_download_dir= self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} info file {zip_download_dir}")


            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-08 13:43:14,850- INFO- common - yaml file config\config.yaml loaded sucessfully]
[2025-12-08 13:43:14,850- INFO- common - yaml file params.yaml loaded sucessfully]
[2025-12-08 13:43:14,850- INFO- common - created a directory artifacts]
[2025-12-08 13:43:14,857- INFO- common - created a directory artifacts/data_ingestion]
[2025-12-08 13:43:14,860- INFO- 1713579087 - Downloading data from https://drive.google.com/file/d/1xJYnY4uHuxdw9IjQ1Ej3g0CP4dspKCqz/view?usp=sharing info file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1xJYnY4uHuxdw9IjQ1Ej3g0CP4dspKCqz
From (redirected): https://drive.google.com/uc?export=download&id=1xJYnY4uHuxdw9IjQ1Ej3g0CP4dspKCqz&confirm=t&uuid=23a06c20-c605-47b4-a1b1-85be1ec46290
To: c:\Users\karakkir\OneDrive - Tietoevry\Kidney-dl\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:02<00:00, 19.3MB/s]

[2025-12-08 13:43:20,513- INFO- 1713579087 - Downloaded data from https://drive.google.com/file/d/1xJYnY4uHuxdw9IjQ1Ej3g0CP4dspKCqz/view?usp=sharing into file artifacts/data_ingestion/data.zip]
